In [1]:
import torch
import torchvision
from torch.utils.data.dataloader import DataLoader
from torchvision.models.video import r3d_18
from data.video import SpatioTemporalDataset

In [2]:
model = r3d_18(pretrained=True)
newmodel = torch.nn.Sequential(*(list(model.children())[:-1]))

In [14]:
preprocessing = torchvision.transforms.Compose([
                torchvision.transforms.Resize(112),
                torchvision.transforms.CenterCrop(112),
                torchvision.transforms.Normalize((0.43216, 0.394666, 0.37645), (0.22803, 0.22145, 0.216989))])

data = SpatioTemporalDataset('/mnt/d/serao/rlvs_cp/train/', transforms=preprocessing, num_clips=16, train=False)
loader = DataLoader(data, batch_size=1, shuffle=True, num_workers=8)

X = []
y = []
limit = len(loader)
for batch_ndx, sample in enumerate(loader):
    out = newmodel(torch.permute(sample[0], (0,2,1,3,4)))
    X.append(out.squeeze().detach().numpy())
    y.append(sample[1].numpy())
    
    if batch_ndx == limit:
        break

In [15]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

clf = make_pipeline(StandardScaler(), LinearSVC(random_state=0, tol=1e-5))
clf.fit(X, y)

/home/serao/violence_detection/violence-detection/env/lib/python3.8/site-packages/sklearn/utils/validation.py:985: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/home/serao/violence_detection/violence-detection/env/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Pipeline(steps=[('standardscaler', StandardScaler()),
                ('linearsvc', LinearSVC(random_state=0, tol=1e-05))])

In [16]:
preprocessing = torchvision.transforms.Compose([
                torchvision.transforms.Resize(112),
                torchvision.transforms.CenterCrop(112),
                torchvision.transforms.Normalize((0.43216, 0.394666, 0.37645), (0.22803, 0.22145, 0.216989))])

test_data = SpatioTemporalDataset('/mnt/d/serao/rlvs_cp/test/', transforms=preprocessing, num_clips=16, train=False)
test_loader = DataLoader(test_data, batch_size=1, shuffle=True, num_workers=8)

limit = len(test_loader)
correct = 0
for batch_ndx, sample in enumerate(test_loader):
    out = newmodel(torch.permute(sample[0], (0,2,1,3,4)))
    predicted = clf.predict(out.squeeze().detach().numpy().reshape(1,-1))
    real = sample[1]
    
    if predicted[0] == real.item():
        correct += 1
    
    if batch_ndx == limit:
        break

print("Accuracy: ", correct/limit)

Accuracy:  0.73
